In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import re
import optuna
from optuna.integration import XGBoostPruningCallback
sns.set_theme(style="darkgrid")

In [2]:
Jordan_df_root=pd.read_csv('Data/크림_조던_제품거래량_최종.csv')

Jordan_df_tmp=pd.DataFrame(Jordan_df_root)
Jordan_df_tmp

,index,사이즈,거래가,거래일,제품한글명,모델번호,출시일,컬러,발매가
0,804.0,275,"623,000원",22/07/28,조던 1 레트로 하이 OG 블랙 모카,555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원"
1,804.0,265,"620,000원",22/07/28,조던 1 레트로 하이 OG 블랙 모카,555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원"
2,804.0,285,"602,000원",22/07/28,조던 1 레트로 하이 OG 블랙 모카,555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원"
3,804.0,270,"760,000원",22/07/28,조던 1 레트로 하이 OG 블랙 모카,555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원"
4,804.0,265,"610,000원",22/07/28,조던 1 레트로 하이 OG 블랙 모카,555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원"
...,...,...,...,...,...,...,...,...,...
185842,4545.0,270,"550,000원",21/04/27,조던 1 레트로 하이 이퀄리티 BHM 2018,AQ7474-001,18/01/16,BLACK/BLACK-WHITE-METALLIC GOLD,"$145 (약 189,300원)"
185843,4545.0,275,"620,000원",21/04/22,조던 1 레트로 하이 이퀄리티 BHM 2018,AQ7474-001,18/01/16,BLACK/BLACK-WHITE-METALLIC GOLD,"$145 (약 189,300원)"
185844,4545.0,250,"440,000원",21/04/17,조던 1 레트로 하이 이퀄리티 BHM 2018,AQ7474-001,18/01/16,BLACK/BLACK-WHITE-METALLIC GOLD,"$145 (약 189,300원)"
185845,4545.0,250,"400,000원",21/03/06,조던 1 레트로 하이 이퀄리티 BHM 2018,AQ7474-001,18/01/16,BLACK/BLACK-WHITE-METALLIC GOLD,"$145 (약 189,300원)"


In [3]:
Jordan_df_tmp['발매가'].fillna(0)
Jordan_df_tmp['발매가'] = Jordan_df_tmp['발매가'].astype(str)

In [4]:
#발매가 모두 원화로 처리
for idx,money in enumerate(Jordan_df_tmp['발매가']):
    if '(' in money:
        Jordan_df_tmp.loc[idx,'발매가'] = re.search(r'[0-9]+,[0-9]+원',money).group()
    else:
        pass

In [5]:
# "235(5Y·W)"와 같은 값 처리
for idx,size in enumerate(Jordan_df_tmp['사이즈']):
    if '(' in size:
        Jordan_df_tmp.loc[idx,'사이즈'] = re.search(r'[0-9]',size).group()
    else:
        pass

In [6]:
#거래가, 발매가를 숫자형 문자열로 표현하기 위해 '원', ','제거 & nan과 '-'은 0으로 대체
Jordan_df_tmp['거래가']=pd.DataFrame(Jordan_df_tmp['거래가'].str.replace('원',''))
Jordan_df_tmp['거래가']=pd.DataFrame(Jordan_df_tmp['거래가'].str.replace(',',''))
Jordan_df_tmp['발매가']=pd.DataFrame(Jordan_df_tmp['발매가'].str.replace('원',''))
Jordan_df_tmp['발매가']=pd.DataFrame(Jordan_df_tmp['발매가'].str.replace(',',''))
Jordan_df_tmp['발매가']=pd.DataFrame(Jordan_df_tmp['발매가'].str.replace('-','0'))
Jordan_df_tmp['발매가']=pd.DataFrame(Jordan_df_tmp['발매가'].str.replace('nan','0'))

In [7]:
#출시일, 거래일도 숫자형 문자열로 표시
Jordan_df_tmp['출시일']=pd.DataFrame(Jordan_df_tmp['출시일'].str.replace('/',''))
Jordan_df_tmp['거래일']=pd.DataFrame(Jordan_df_tmp['거래일'].str.replace('/',''))

In [8]:
# df.groupby(그룹이되는컬럼)[계산하고 싶은 컬럼].집계함수()
#제품별 거래량 파악
Jordan_df_tmp['총거래량']=0
Jordan_df_tmp.groupby('제품한글명')['총거래량'].count()
Jordan_df_tmp=pd.merge(Jordan_df_tmp,Jordan_df_tmp.groupby('제품한글명')['총거래량'].count(),on='제품한글명',how='left').drop('총거래량_x',axis=1).rename(columns={'총거래량_y':'총거래량'})

In [9]:
#해당 컬럼값들 숫자형으로 변환
Jordan_df_tmp['거래가'] = pd.to_numeric(Jordan_df_tmp['거래가'])
Jordan_df_tmp['발매가'] = pd.to_numeric(Jordan_df_tmp['발매가'])
Jordan_df_tmp['거래일'] = pd.to_numeric(Jordan_df_tmp['거래일'])
Jordan_df_tmp['사이즈'] = pd.to_numeric(Jordan_df_tmp['사이즈'])

In [10]:
#발매가가 없는 제품은 제품 거래가의 평균으로 대체
product_price = Jordan_df_tmp.groupby('제품한글명')['거래가'].mean()

for i in tqdm(range(len(Jordan_df_tmp))):
    if Jordan_df_tmp['발매가'][i] == 0:
        product_name=Jordan_df_tmp['제품한글명'][i]
        Jordan_df_tmp['발매가'][i] = product_price.loc[product_name]

 65%|██████████████████████████████████████████████▎                        | 121068/185847 [00:01<00:00, 73110.89it/s]C:\Users\gi257\AppData\Local\Temp/ipykernel_2756/3798536552.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Jordan_df_tmp['발매가'][i] = product_price.loc[product_name]
100%|███████████████████████████████████████████████████████████████████████| 185847/185847 [00:07<00:00, 25304.58it/s]


In [11]:
#출시일이 없는 제품은 처음 이뤄진 거래일로 대체
for i in tqdm(range(len(Jordan_df_tmp))):
    if Jordan_df_tmp['출시일'][i] == '-':
        Jordan_df_tmp['출시일'][i] = sorted(Jordan_df_tmp[Jordan_df_tmp['제품한글명']==Jordan_df_tmp['제품한글명'][i]]['거래일'])[0]
Jordan_df_tmp['출시일'] = pd.to_numeric(Jordan_df_tmp['출시일'])

  8%|█████▉                                                                  | 15191/185847 [00:00<00:02, 78535.43it/s]C:\Users\gi257\AppData\Local\Temp/ipykernel_2756/770170754.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Jordan_df_tmp['출시일'][i] = sorted(Jordan_df_tmp[Jordan_df_tmp['제품한글명']==Jordan_df_tmp['제품한글명'][i]]['거래일'])[0]
100%|████████████████████████████████████████████████████████████████████████| 185847/185847 [01:14<00:00, 2510.53it/s]


In [12]:
#사이즈에 오류값(size=2)이 있는 경우 사이즈를 랜덤하게 입력
import random
rand_size = [200,205,210,215,220,225,230,235,240,245,250] #GS여서 250까지만 제한
for i in tqdm(range(len(Jordan_df_tmp))):
    if Jordan_df_tmp['사이즈'][i] == 2:
        Jordan_df_tmp['사이즈'][i] = random.choice(rand_size)

  0%|                                                                                       | 0/185847 [00:00<?, ?it/s]C:\Users\gi257\AppData\Local\Temp/ipykernel_2756/1749332734.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Jordan_df_tmp['사이즈'][i] = random.choice(rand_size)
 22%|████████████████▌                                                         | 41680/185847 [03:09<10:56, 219.74it/s]


KeyboardInterrupt: 

In [13]:
Jordan_df_tmp['컬러']=Jordan_df_tmp['컬러'].apply(str) #컬러에 float변수가 존재하여 문자형으로 형변환

#자주 등장하는 컬러로 색상정보 대체
color_ex=['RED', 'ORANGE', 'STARFISH', 'YELLOW', 'GREEN', 'BLUE', 'CORAL', 'PINK', 'PURPLE','GREY', 'SILVER', 'BLACK', 'WHITE', 'MOCHA', 'STEALTH', 'SAIL', 'NAVY', 'BROWN', 'WHEAT', 'BORDEAUX']
color_list=[]
for color in tqdm(Jordan_df_tmp['컬러']):
    # 구분자로 구분
    color_list.append(re.split('[/-]', color.replace(' ','/')))

filtered_color_list=[]
for words in color_list:
    filtered_words=[]
    for word in words:
        if word in color_ex:
            filtered_words.append(word)
    filtered_color_list.append(list(set(filtered_words))) #중복제거
    
Jordan_df_tmp['컬러테스트']=filtered_color_list

#기존의 '컬러' 열을 '컬러테스트'열로 대체
Jordan_df_tmp['컬러']=Jordan_df_tmp['컬러테스트']
Jordan_df_tmp=Jordan_df_tmp.drop(columns='컬러테스트')

100%|██████████████████████████████████████████████████████████████████████| 185847/185847 [00:01<00:00, 165194.60it/s]


In [20]:
color_test.to_csv('Data/크림_조던_제품거래량_전처리_최종.csv', index = False)